In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from astropy import units as U
from astropy import constants as const
from astropy.coordinates import SkyCoord
import astropy.units as U

In [17]:
observation_metadata = pd.read_csv('all_alma_data.csv')
observation_metadata = observation_metadata[['s_resolution', 't_exptime', 'em_min', 't_resolution',
                        'em_max', 'em_resolution', 'spatial_resolution', 'bandwidth', 
                        'frequency', 'frequency_support', 'velocity_resolution', 'sensitivity_10kms', 
                        'cont_sensitivity_bandwidth', 'scan_intent', 's_ra', 's_dec']]
observation_metadata = observation_metadata.dropna()

scan_intents = observation_metadata['scan_intent'].values
ids = []
for i, scan in enumerate(scan_intents):
    if 'TARGET' in scan:
        ids.append(i)
ids = np.array(ids)
print(scan_intents.shape, ids.shape)

observation_metadata = observation_metadata.iloc[ids, :]
observation_metadata = observation_metadata[(observation_metadata['s_resolution'] == observation_metadata['spatial_resolution'])]
observation_metadata = observation_metadata.drop(columns=['s_resolution'])     
observation_metadata = observation_metadata[['spatial_resolution', 't_exptime', 'em_min','em_max', 'em_resolution', 
            'bandwidth', 'frequency', 'frequency_support', 'velocity_resolution', 's_ra', 's_dec', 't_resolution']]
observation_metadata['velocity_resolution'] = observation_metadata['velocity_resolution'] * 10**(-3)

observation_metadata = observation_metadata.rename(columns={'spatial_resolution': 'spatial_resolution [arcsec]',
                                                            't_exptime': 'integration_time [s]',
                                                            'em_resolution': 'frequency_resolution [m]',
                                                            'bandwidth': 'bandwidth [MHz]',
                                                            'frequency': 'frequency [Ghz]',
                                                            'velocity_resolution': 'velocity_resolution [Km/s]',
                                                            'em_min': 'frequency_min [m]', 'em_max': 'frequency_max [m]',
                                                            's_ra': 'ra [deg]', 's_dec': 'dec [deg]'})
freq_sup = observation_metadata['frequency_support'].values
observation_metadata['bandwidth [MHz]'] = observation_metadata['bandwidth [MHz]'].values * 10**(-6)
freq_res = [int(float(f.split(',')[1][:-3]))* 10 ** (-3)  for f in freq_sup]

observation_metadata.info()

coords = SkyCoord(ra=observation_metadata['ra [deg]'], dec=observation_metadata['dec [deg]'], unit='deg').to_string('hmsdms')
b = 'J2000 '
coords = [b + i for i in coords]
observation_dist = observation_metadata[['spatial_resolution [arcsec]', 'integration_time [s]', 
                                        'velocity_resolution [Km/s]', 'ra [deg]', 'dec [deg]', 'bandwidth [MHz]']]
observation_dist['J2000 coordinates'] = coords
observation_dist['frequency_resolution [MHz]'] = freq_res
observation_dist.to_csv('obs_configurations.csv', index=False)


/var/folders/66/76q50s015tq8wc5qz1pvg7y40000gn/T/ipykernel_1122/55381802.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  observation_metadata = pd.read_csv('all_alma_data.csv')


(81999,) (37906,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 35089 entries, 4 to 81998
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   spatial_resolution [arcsec]  35089 non-null  float64
 1   integration_time [s]         35089 non-null  float64
 2   frequency_min [m]            35089 non-null  float64
 3   frequency_max [m]            35089 non-null  float64
 4   frequency_resolution [m]     35089 non-null  float64
 5   bandwidth [MHz]              35089 non-null  float64
 6   frequency [Ghz]              35089 non-null  float64
 7   frequency_support            35089 non-null  object 
 8   velocity_resolution [Km/s]   35089 non-null  float64
 9   ra [deg]                     35089 non-null  float64
 10  dec [deg]                    35089 non-null  float64
 11  t_resolution                 35089 non-null  float64
dtypes: float64(11), object(1)
memory usage: 3.5+ MB


/var/folders/66/76q50s015tq8wc5qz1pvg7y40000gn/T/ipykernel_1122/55381802.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observation_dist['J2000 coordinates'] = coords
/var/folders/66/76q50s015tq8wc5qz1pvg7y40000gn/T/ipykernel_1122/55381802.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observation_dist['frequency_resolution [MHz]'] = freq_res


In [19]:
np.unique(observation_dist['bandwidth [MHz]'])

array([  58.59375,   62.5    ,  117.1875 ,  125.     ,  234.375  ,
        250.     ,  468.75   ,  500.     ,  937.5    , 1000.     ,
       1875.     , 1937.5    , 1992.1875 , 2000.     ])

In [ ]:
obs_params = pd.read_csv('obs_configurations.csv')
obs_params.info()

In [ ]:

plt.hist(freq_res)

In [ ]:
observation_dist['s_ra'].values[:10]

In [ ]:
c = const.c.to(U.km / U.s).value
print(c)
restfreq = 1.420405752
restvel = c * ((restfreq - observation_metadata['frequency [GHz]']) / restfreq)
observation_metadata['restvel'] = restvel

In [ ]:
sns.set_style('whitegrid')
sns.set_context('talk')
sns.set_palette('colorblind')


for column in observation_metadata.columns:
    fig, ax = plt.subplots(figsize=(5, 5))
    sns.histplot(x=column, data=observation_metadata, kde=False, ax=ax)
    plt.show()
    print(np.mean(observation_metadata[column]), np.std(observation_metadata[column]))
